In [56]:
# Importation des bibliothèques nécessaires
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [57]:
# Lien de la page à scraper
url='https://www.booking.com/searchresults.fr.html?ss=%C3%8Ele-de-France&ssne=%C3%8Ele-de-France&ssne_untouched=%C3%8Ele-de-France&efdco=1&label=msn-cmvrz2iIJvoSCIe32hC*fA-79989658705793%3Atikwd-79989834233190%3Aloc-66%3Aneo%3Amte%3Alp125942%3Adec%3Aqsbooking&sid=c79f1bdefeb976499d3644dcf3e1d5fe&aid=2369661&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=1425&dest_type=region&checkin=2024-06-08&checkout=2024-06-09&group_adults=2&no_rooms=1&group_children=0'
# Téléchargement du contenu de la page
response = requests.get(url)

In [60]:
# Vérification du fonctionnement de la page
response

# Response [200] : La page a été téléchargée avec succès

<Response [200]>

In [74]:
# Analyse du code HTML
soup = BeautifulSoup(response.content, 'html.parser')

In [75]:
# Recherche des éléments d'hôtels, limité à 25 hôtels
elements = soup.find_all('div', class_="c82435a4b8 a178069f51 a6ae3c2b40 a18aeea94d d794b7a0f7 f53e278e95 c6710787a4", limit=25)

In [77]:
# Liste vide pour stocker les données extraites
Data = []

for element in elements:
    # Extraction du titre de l'hôtel
    title = element.find('div', class_='f6431b446c a15b38c233').text.strip()

    # Extraction des évaluations clients
    Retour_tag = element.find('div', class_='a3b8729ab1 e6208ee469 cb2cbb3ccb')
    Retour = Retour_tag.text.strip('()') if Retour_tag  else 'Info non disponible'

    # Extraction de la note des clients
    Note_tag = element.find('div', class_='a3b8729ab1 d86cee9b25')
    Note = Note_tag.get_text(separator='').strip().split('Avec')[0].strip()

    # Extraction du nombre d'expériences vécues
    Nbr_exp_tag = element.find('div', class_='abf093bdfe f45d8e4c32 d935416c47')
    Nbr_exp = Nbr_exp_tag.get_text(separator='').strip().split('expériences')[0].strip()

    # Extraction de l'adresse de l'hôtel
    Adress_tag = element.find('span', class_='aee5343fdb def9bc142a')
    Adress = Adress_tag.text.strip() if Adress_tag else 'Info non disponible'
    
    # Extraction du nombre d'étoiles de l'hôtel
    Etoile_div = element.find('div', class_='b3f3c831be')
    Etoile = Etoile_div['aria-label'] if Etoile_div else 'Info non disponible'
    
    # Extraction du certificat de l'hôtel
    Certificat_tag = element.find('span', class_='abf093bdfe e6208ee469 f68ecd98ea')
    Certificat = Certificat_tag.text.strip() if Certificat_tag else 'Pas de certificat'
 
    # Ajout des données dans la liste
    Data.append({'Titre': title,
                 'Evaluation': Retour ,
                 'Note des clients ': Note ,
                 'Nombre expéreinces vécus' : Nbr_exp ,
                 'Adresse': Adress ,
                 'Étoile': Etoile ,
                 'Certficat': Certificat
                })
    
    # Création d'un DataFrame pandas à partir des données
    df = pd.DataFrame(Data)
    
    # Enregistrement du DataFrame au format Excel
    df.to_excel('hotels_Ile_De_France_data.xlsx', index=False)